### Import libraries

In [54]:
# === Step 1: Import Libraries ===
import pandas as pd
import numpy as np
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
from sklearn.linear_model import LinearRegression
import socket

### Customize

In [55]:
# Customize parameters
my_input_file='0407 my_input.csv'
my_host='127.0.0.8'

# define regression paraneters 
my_x_values = ['Utilization', 'Downtime', 'Energy_Consumption', 'Defect_Rate']
my_y_value = 'Failure_Risk'

# Seconds to refresh dashboard
my_refresh=10

### Load and train model on initial data

In [56]:
# Train ML model once
training_data = pd.read_csv(my_input_file)
X_train = training_data[['Utilization', 'Downtime', 'Energy_Consumption', 'Defect_Rate']]
y_train = training_data['Failure_Risk']
model = LinearRegression().fit(X_train, y_train)

### Function to create dummy data

In [57]:
# Counting the number of machines to generate future data
n = training_data['Machine_ID'].nunique() 

# === Step 2: Simulate Base Data + Train ML Model ===
machine_ids = [f'M{i+1}' for i in range(n)]

def generate_data():
    return pd.DataFrame({
        'Machine_ID': machine_ids,
        'Utilization': np.random.randint(60, 100, size=n),
        'Downtime': np.random.randint(1, 12, size=n),
        'Energy_Consumption': np.random.randint(100, 170, size=n),
        'Defect_Rate': np.round(np.random.uniform(1.0, 5.0, size=n), 2),
        'Failure_Risk': np.round(np.random.uniform(0.05, 0.6, size=n), 2)
    })



### Setup dashboard app

In [58]:
# Build Dash App
app = Dash(__name__)

app.layout = html.Div([
    html.H2("Role-Based KPI Dashboard"),

    html.Label("Select Stakeholder Role:"),
    dcc.Dropdown(
        id='role_selector',
        options=[
            {'label': 'Executive', 'value': 'executive'},
            {'label': 'Operations Manager', 'value': 'operations'},
            {'label': 'Maintenance Head', 'value': 'maintenance'}
        ],
        value='executive',
        clearable=False
    ),

    dcc.Interval(id='update_interval', interval=my_refresh*1000, n_intervals=0),
    html.Div(id='role_specific_output')
])


### Callback with live data

In [59]:

# Callback Based on Role and Interval Updates 
@app.callback(
    Output('role_specific_output', 'children'),
    Input('role_selector', 'value'),
    Input('update_interval', 'n_intervals')
)
def update_dashboard(role, n):
    df = generate_data()
    X_live = df[['Utilization', 'Downtime', 'Energy_Consumption', 'Defect_Rate']]
    df['Predicted_Risk'] = model.predict(X_live)

    if role == 'executive':
        # Executive: Summary KPIs and trends
        avg_risk = round(df['Predicted_Risk'].mean(), 3)
        avg_util = round(df['Utilization'].mean(), 2)

        fig = px.bar(df, x='Machine_ID', y='Predicted_Risk', title='Avg Failure Risk by Machine', color='Predicted_Risk')
        return html.Div([
            html.H4(f"Executive Summary"),
            html.P(f"Average Utilization: {avg_util}%"),
            html.P(f"Average Predicted Failure Risk: {avg_risk}"),
            dcc.Graph(figure=fig)
        ])

    elif role == 'operations':
        # Operations: Live Utilization, Downtime, Defect
        fig1 = px.bar(df, x='Machine_ID', y='Utilization', title='Real-Time Machine Utilization', color='Predicted_Risk')
        fig2 = px.bar(df, x='Machine_ID', y='Defect_Rate', title='Defect Rate by Machine', color='Predicted_Risk')
        return html.Div([
            html.H4("🔧 Operational View"),
            dcc.Graph(figure=fig1),
            dcc.Graph(figure=fig2)
        ])

    elif role == 'maintenance':
        # Maintenance: Risk-Ranked Machine Table
        df_sorted = df.sort_values(by='Predicted_Risk', ascending=False).reset_index(drop=True)
        table_html = html.Table([
            html.Tr([html.Th(col) for col in df_sorted.columns])
        ] + [
            html.Tr([html.Td(df_sorted.iloc[i][col]) for col in df_sorted.columns])
            for i in range(len(df_sorted))
        ])
        return html.Div([
            html.H4("Maintenance Priority List (Sorted by Risk)"),
            table_html
        ])


# Display Dash server address
ip_address = socket.gethostbyname(socket.gethostname())
print(f"Dash app running at http://{ip_address}:8050")

if __name__ == '__main__':

    # For external access prefix the first option  with # and remove the # from the second option
    app.run(my_host, port=8050, debug=True)
    #app.run(host='0.0.0.0', port=8050, debug=True))

Dash app running at http://192.168.1.73:8050
